# **Introduction**

This tutorial will introduce you to a new method of learning --- Ensemble learning, particularly focusing on algorithms behind Boosting and Bagging. The idea of ensemble learning is to build a predictive model by combining a collection of simpler base models, also known as weak learners or base learners.
It is well known that an ensemble learning is usually more accurate than a single learner, and ensemble methods have already achieved great success in many real-world problems.



## Tutorial content

In this tutorial, we will show two types of ensemble learning: Bagging and Boosting. And discuss in depth about popular model in each type: Random Forest and Adaboost, including the algorithms behind them and how to use them in Python. This tutorial is designed to focus on the algorithm side and conceptional side. The coding part is just a good support of a better understanding. This tutorial also includes a practice question about the iteration of Adaboost to deepen your knowledge. Are you ready to start the journey?

**Content**:
* <a href = '#Bagging'>Bagging</a>
    * <a href = '#General'>General</a>
    * <a href = '#Coding'>Coding</a>
    * <a href = '#Random_Forest'>Random Forest</a>
    * <a href = '#Algorithm_of_Random_Forest'>Algorithm of Random Forest</a>
* <a href = '#Boosting'>Boosting</a>
    * <a href = '#Regression'>Regression</a>
    * <a href = '#Classification'>Classification</a>
    * <a href = '#Algorithm_of_Adaboost'>Algorithm of Adaboost</a>
    * <a href = '#Practice'>Practice</a>
    * <a href = '#Coding'>Coding</a>
* <a href = '#Bagging_vs_Boosting'>Bagging_vs_Boosting</a>
* <a href = '#References'>References</a>
* <a href = '#Useful_Resources'>Useful_Resources</a>  

# **Bagging**
## General
The name "Bagging" came from the abbreviation of Bootstrap Aggregation [Breiman, 1996]. As we could tell from the name itself, the two key elements of this method are bootstrap and aggregation. It is a general-purpose method to reduce the variance of a learning method by averaging together multiple estimates. 

It applies bootstrapping to gain data subsets for training the base learners. To be specific, it takes repeated samples from the training set **with replacement**, and repreat this process B times to generate B different, independently generated bootstrapped training sets. We will discuss it more in the context of decision trees.

To apply bagging to regression trees, we can construct B regression trees using B bootstrapped training sets mentioned above, and then average these predictions according to the equation:
\begin{gather*} 
f_{avg}(x) = \frac{1}{B} \sum_{b=1}^{B}f_b(x)    
\end{gather*}
Notice we would use avg(f) for model aggregation if it is a regression and majority voting if it is a classification. Since each part of the ensemble could be done independently, we call it a **Parallel model training.**
## Coding
Next, Let's get hands dirty and have a practice to prove that Bagging has better performance than single learner with the following Wine classification problem. Let us suppose that we decide to use the features [ Alcohol, OD280/OD315 of diluted wines] to classify the Class label of wine after feature selection (we skip this part since it is not related to our topic today).

**Data load and preparation**

In [56]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/'
'machine-learning-databases/wine/wine.data',header=None)
df.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
'Alcalinity of ash', 'Magnesium', 'Total phenols',
'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
'Proline']

df = df[df['Class label'] != 1]
y = df['Class label'].values
X = df[['Alcohol', 'OD280/OD315 of diluted wines']].values

In [47]:
df.head()

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
59,2,12.37,0.94,1.36,10.6,88,1.98,0.57,0.28,0.42,1.95,1.05,1.82,520
60,2,12.33,1.10,2.28,16.0,101,2.05,1.09,0.63,0.41,3.27,1.25,1.67,680
61,2,12.64,1.36,2.02,16.8,100,2.02,1.41,0.53,0.62,5.75,0.98,1.59,450
62,2,13.67,1.25,1.92,18.0,94,2.10,1.79,0.32,0.73,3.80,1.23,2.46,630
63,2,12.37,1.13,2.16,19.0,87,3.50,3.10,0.19,1.87,4.45,1.22,2.87,420


**Splitting data and model setting**

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
lbencoder = LabelEncoder()
y = lbencoder.fit_transform(y)
X_train, X_test, y_train, y_test =train_test_split(X, y,
                                                   test_size=0.25,
                                                   random_state=1,
                                                   stratify=y)
tree = DecisionTreeClassifier(criterion='entropy',
                              max_depth=None,
                              random_state=1)
bc = BaggingClassifier(base_estimator=tree,
                       n_estimators=300,
                       max_samples=1.0,
                       max_features=1.0,
                       bootstrap=True,
                       bootstrap_features=False,
                       n_jobs=1,
                       random_state=1)

**Result Comparison**

In [49]:
from sklearn.metrics import accuracy_score
tree = tree.fit(X_train, y_train)
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)
tree_train = accuracy_score(y_train, y_train_pred)
tree_test = accuracy_score(y_test, y_test_pred)

bc = bc.fit(X_train, y_train)
y_train_pred = bag.predict(X_train)
y_test_pred = bag.predict(X_test)
bag_train = accuracy_score(y_train, y_train_pred)
bag_test = accuracy_score(y_test, y_test_pred)
print('Decision tree test accuracies %.4f'% (tree_test))
print('Bagging test accuracies %.4f'% (bag_test))


Decision tree test accuracies 0.8333
Bagging test accuracies 0.9000


## Random_Forest
While, sometimes, it is hard for bootstrapping to create many uncorrelated datasets. And highly correlated quantities do not lead to as large of a reduction in variance as compared to averaging many uncorrelated ones. So, to solve issue, we will introduce an extension of Bagging --- **Random Forest.**

Random Forest (RF) [Breiman, 2001] is a representative of the state-of-art ensemble methods. It provides an improvement over bagged trees by way of a small tweak that achieves a decorrelation of the trees.

## Algorithm_of_Random_Forest
A random smaple of p features is selected **at each split** and the one out of only those p with the largest Gain is used to make the split. We draw a bootstrap dataset of size n from training data D. Then to build decision tree, we will recursively repeat the following splitting steps for each terminal/leaf node: 
1. Select p features from the original d features in D ramdomly. 
2. Pick the best feature among the p to split on.
3. Split the node into descendant terminal nodes.

The full logic is as following:


**input**: Data set D = {(x1,y1),(x2,y2)....(xm,ym)}; Feature subset size K.

**Process**:

1.  create a tree node based on D --> N;
2.  if all instances in the same class then return N;
3.  the set of features that can be split further --> F;
4.  if F is empty then return N;
5.  select K features from F randomly --> F';
6.  the feature which has the best split point in F' --> N.f;
7.  the best split point on N.f --> N.p;
8.  subset of D with values on N.f smaller than N.p --> Dl;
9.  subset of D with values on N.f larger than or equal to N.p --> Dr;
10.  call the process with parameters(Dl, K) --> Nl;
11.  call the process with parameters(Dr, K) --> Nr;
12.  return N

**output**: A random decision tree

# **Boosting**
Boosting is another ensemble method for improving the predictive performance. Instead of being a Parallel model training method, Boosting is **Sequential model training** process. The idea of boosting is to train many weak learners that learn to do well at different parts of the input space. You might ask what is different parts of input? The answer is: Parts at which trained weak learners make mistakes **previously**.

## Regression
Let us first talk about the Boosting in regression. Instead of fitting a single large decision tree, Boosting approach learns slowly iteratively (from previous steps).

Given the current model from the previous step, we fit a regression model to the residuals from it as the response values, rather than the original response values. Then we add this new tree into the fitted function and update the residuals. We repeat this process for B times. The steps in details are as follows:
1. Set f(x) = 0 and ri = yi for all instances i in training set D=(X,y)
2. for b = 1,2,,,,B:
    1. Fit a regression tree Tb to (X,r), regularized via a h_max or n_min (both are hyper-parameters). Denote the predictions of Tb on an instance x by fb(x).
    2. Update prediction model f(x) + lambda * fb(Xi) --> f(x)
    3. Update the residuals: ri - lambda * fb(xi) --> ri
    
    
Output the boosted regression model 
\begin{gather*} 
f(x) = \sum_{b=1}^{B} \lambda f_b(x)    
\end{gather*}

## Classification
Now let us discuss about how the idea of boosting could be applied to classification problems. Like we discussed a popular model of Bagging, we will also introduce you a super star of boosting --- **Adaboost** [Freund & Schapire, 1997].


The idea is to repeatedly train classifier on modified versions of the data, thereby producing a sequence of weak classifiers f1, f2, ,,fB : x-->{-1,1}. The predictions from all these weak learners are combined through a **weighted** majority vote. At each iteration of the boosting, the **weight for a weak learner** is computed as a function that is inversely proportional to its training error(TE). On the other side, the **weight for a strong learner** is computed as a function that is proportional to its training error(TE).

## Algorithm_of_Adaboost
**Input**: data set D={(x1,y1),(x2,y2)....(xm,ym)};
    Base learning algorithm L;
    Number of learning rounds T.
**Process**:
1. D1(x) = 1/m  Start with the same weight
2. for t = 1,2,,,,T:
    3. ht = L(D,Dt);    Train a classifier ht from D under distribution Dt
    4. et = Px~dt(ht(x) != f(x));    Evaluate the error of ht
    5. if et > 0.5 then break;
    6. alpha(t) = 0.5 * ln((1 - et) / et)    Determine the weight of ht
    7. D_{t+1}(x) = Dt(x) / Zt    Update the weight of learners, where Zt is a normalization factor.
8. end
**output**: 
\begin{gather*} 
H(x) = sign(\sum_{t=1}^{T} \alpha_t h_t(x))   
\end{gather*}

## Practice
Here, we add a practice to deepen your understanding of the algorithm behind Adaboost.

**Question**: we will apply Adaboost to a toy dataset for classification. Consider the 2-d dataset in Figure 1. 

The dataset consists of the following 4 points: (x1, y1) : (<0,−1>, −), (x2, y2) :(<1, 0>, +), (x3, y3) : (<−1, 0>, +), (x4, y4) : (<0, 1>, −).Having decided to use simple decision stumps as weak classifiers, show how Adaboost works on this dataset for B = 4 iterations. For each  step, remember to compute quantities listed below, and fill in the corresponding entries in the table.
\begin{gather*} 
\epsilon_b, \alpha_b, \beta_i^{b-1}, f_b^{(x_i)}  
\end{gather*}

Note that at each iteration, there could be multiple decision stumps with same classification error. You can select the best learner at each iteration randomly. Draw your weak classifier that is learned on each iteration. For example, the selected learner f1 at b = 1 is shown in Figure 1 (b).

**Figure 1**:
<img src="1.png" width=500 height=150>
**table to be filled**:
<img src="table.png" width=500 height=150>

**Solution**:
Did you solve the problem and fill the table above? Here is the solution for you to check!
<img src="solution.jpg" width=500 height=150>
<img src="table2.png" width=500 height=150>


Let us take the iteration 1 as example. At iteration 1, epsilon_b = 1/4 = 0.25. Then we can compute the alpha_b = ln(0.75/0.25)/2 = 0.549  

Then we can compute beta_i according to this alpha_b. And don't forget to normalize them so that the sum of weights is equal to 1.

Then move to iteration 2, there is still one error. And we could gain the new error rate by checking its weight gained from previous iteration.

If you repeat this process, you should have gained the same answer as solution! Well Done!

## Coding

In [57]:
from sklearn.ensemble import AdaBoostClassifier
tree = DecisionTreeClassifier(criterion='entropy',
                              max_depth=1,
                              random_state=1)
ac = AdaBoostClassifier(base_estimator=tree,
                         n_estimators=300,
                         learning_rate=0.1,
                         random_state=1)


In [58]:
# Decision tree training
tree = tree.fit(X_train, y_train)
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)
tree_train = accuracy_score(y_train, y_train_pred)
tree_test = accuracy_score(y_test, y_test_pred)

# Adaboost training
ac = ac.fit(X_train, y_train)
y_train_pred = ada.predict(X_train)
y_test_pred = ada.predict(X_test)
ada_train = accuracy_score(y_train, y_train_pred)
ada_test = accuracy_score(y_test, y_test_pred)

# Results Printing
print('Decision tree test accuracies %.3f'% (tree_test))
print('AdaBoost test accuracies %.3f'% (ada_test))


Decision tree test accuracies 0.867
AdaBoost test accuracies 0.900


# Bagging_vs_Boosting
We will conclude this tutorial with a qualitative comparison of Bagging and Boosting methods.
- Bagging creates training sets by bootstrapping, whereas Boosting(classification) reweights the trainning instances.
- Bagging is a parallel model training, whereas Boosting is a sequential model training.
- The weight of each learner is the same for Bagging, whereas weights in Boosting change based on their previous performances.
- Bagging could reduce variance, whereas Boosting tend to reduce bias and variance at the same time.

**You made it! Nice Job!** It is pleasure to be with you in this trip and talk about basic knowledge and algorithms behind ensemble learnings. I also provide my references for this tutorial and some other materials you might be interested in.

# References
1. Zhi-Hua Zhou, "Ensemble Methods: Foundations and Algorithms", CRC Press, 2012
2. Trevor Hastie, Robert Tibshirani, "An Introduction to Statistical Learning" Chapter 8.2
3. Trevor Hastie, Robert Tibshirani, Jerome Friedman, "Elements of Statistical Learning" Section 10.1
4. Practice question: Upenn CIS520 Slides. https://canvas.upenn.edu/courses/1278549/files/55117366/download?verifier=bmvXrMQnOW9ji4DydSzxr7FnUNbbiwQzzRzbSl2Y&wrap=1
5. Scikit Learn Ensemble Guide
6. Kaggle Ensembling Guide

# Useful_Resources
1. Zhi-Hua Zhou, "Ensemble Methods: Foundations and Algorithms". https://tjzhifei.github.io/links/EMFA.pdf
2. Trevor Hastie, Robert Tibshirani, "An Introduction to Statistical Learning". https://www.statlearning.com/
3. Scikit Learn Ensemble Guide.  https://scikit-learn.org/stable/modules/ensemble.html
4. Kaggle Ensembling Guide. https://mlwave.com/kaggle-ensembling-guide/
5. Kaggle Winning Ensemble. https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/14335
6. github page about ensemble learning: https://github.com/vsmolyakov/experiments_with_python/blob/master/chp01/ensemble_methods.ipynb
7. Ensemble Learning to Improve Machine Learning Results. https://blog.statsbot.co/ensemble-learning-d1dcd548e936